In [1]:
#Setup
import pandas as pd
import numpy as np

#Reading Data
isi_auftragsdetails = pd.read_csv("../data/ISI_Daten/ISI_Auftragsdetails_s822.csv", sep="|")
isi_auftraege = pd.read_csv("../data/ISI_Daten/ISI_Auftraege.csv", sep="|")
isi_aufenthalt = pd.read_csv("../data/ISI_Daten/ISI_FzAufenthalte.csv", sep="|")
isi_stamm = pd.read_csv("../data/ISI_Daten/ISI_Stammdaten.csv", sep="|")

/home/dbpjs/anaconda3/envs/pjs/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
##############################################################################
#Bus-IDs filtern:
##############################################################################

In [3]:
# isi_auftragsdetails = isi_auftragsdetails['FZ_NR'].unique()
# ids_aufenthalt = isi_aufenthalt['FZ-ID'].unique()
# ids_stamm = isi_stamm['FZ-ID'].unique()

# common_ids = list(set(ids_auftrag) & set(ids_aufenthalt) & set(ids_stamm))
# common_ids =list(map(str, common_ids))

#all_ids = pd.read_csv("../Cleaned_Data/combined_ids.csv")

#all_ids["Spalte"]=all_ids[0:]

#isi_auftragsdetails = isi_auftragsdetails[~isi_auftragsdetails["FZ_NR"].isin(all_ids["Spalte"])]
#isi_aufenthalt = isi_aufenthalt[~isi_aufenthalt['FZ-ID'].isin(all_ids["Spalte"])]
#isi_stamm = isi_stamm[~isi_stamm['FZ-ID'].isin(all_ids["Spalte"])]
#isi_auftraege = isi_auftraege[~isi_auftraege['FZ_NR'].isin(all_ids["Spalte"])]

In [7]:
##############################################################################
#Clean Auftragsdetails
##############################################################################

In [8]:
#Duplikate entfernen
isi_auftragsdetails = isi_auftragsdetails.drop_duplicates()

#Spaltennamen anpassen
isi_auftragsdetails = isi_auftragsdetails.rename(columns={'FZ_NR': 'FZ-ID'})

#Unnötige Spalten löschen
isi_auftragsdetails = isi_auftragsdetails.drop(['WERK'], axis=1)

In [9]:
##############################################################################
#Clean Aufenthalt
##############################################################################v

In [10]:
#Duplikate entfernen
isi_aufenthalt = isi_aufenthalt.drop_duplicates()

#WHO-Nummer umrechnen
isi_aufenthalt['WHO_NUMMER'] = isi_aufenthalt['WHO_NUMMER'] - 1000000000

#Unnötige Spalten entfernen
isi_aufenthalt = isi_aufenthalt.drop(['FZ_TYP', 'HZG', 'DAT_WERKSBEZUG', 'PLAN_ÜBERGABE_1', 'PLAN_ÜBERGABE_2'], axis=1)

#Timestamp "DAT_AUSBUCH" zu Integer wechseln
isi_aufenthalt["DAT_AUSBUCH"] = isi_aufenthalt.apply(
    lambda row: 0 if np.isnan(row['DAT_AUSBUCH']) else row['DAT_AUSBUCH'],
    axis=1)

isi_aufenthalt["DAT_AUSBUCH"] = isi_aufenthalt["DAT_AUSBUCH"].astype(int)

In [11]:
##############################################################################
#Clean Stammdaten
#############################################################################

In [12]:
#Duplikate entfernen
isi_stamm = isi_stamm.drop_duplicates()

#Busmaße in Meter umwandeln
isi_stamm['FZ_LAENGE'] = isi_stamm['FZ_LAENGE'] / 1000
isi_stamm['FZ_BREITE'] = isi_stamm['FZ_BREITE'] / 1000
isi_stamm['FZ_HOEHE'] = isi_stamm['FZ_HOEHE'] / 1000

#Unnötige Spalten löschen
isi_stamm = isi_stamm.drop(['WERKSTATT_NAME', 'CHASSIS_BAUART', 'D2_TYP', 'KRAFTSTOFF_ART', 'NENNLEISTUNG', 'MARKE', 'ZULASSUNGSDATUM', 'REISEGESCHWINDGK'], axis=1)

#Plätze kombinieren und alte Spalten löschen
isi_stamm['PLAETZE'] = isi_stamm['SITZPLAETZE'] + isi_stamm['STEHPLAETZE']
isi_stamm = isi_stamm.drop(['SITZPLAETZE','STEHPLAETZE' ], axis=1)

#Inaktive Fahrzeuge entfernen
isi_stamm = isi_stamm[isi_stamm['AKTIV'] != 0]

#Klima kategorisieren
isi_stamm['KLIMAANLAGE'] = isi_stamm['KLIMAANLAGE'].astype('category').cat.codes
isi_stamm['KLIMAANLAGE'] = isi_stamm['KLIMAANLAGE'] + 1

In [13]:
##############################################################################
#Clean Auftrage
#############################################################################

In [15]:
#Duplikate entfernen
#isi_auftraege = isi_auftraege.drop_duplicates()

#Spaltennamen anpassen
isi_auftraege = isi_auftraege.rename(columns={'FZ_NR': 'FZ-ID'})
isi_auftraege = isi_auftraege.rename(columns={'AUFTRAG': 'AUFTRAGSNR'})

#WHO-Nummer umrechnen
isi_auftraege['WHO_NUMMER'] = isi_auftraege['WHO_NUMMER'] + 1000000000

#Unnötige Spalten entfernen
isi_auftraege = isi_auftraege.drop(['WERK', 'AUSGANG', 'SYSTEMSTATUS', 'ARBEITSPLATZ', 'BAHNSTELLE'], axis=1)

#Timestamp "UPDATE_DATE" zu Integer wechseln
# isi_aufenthalt["UPDATE_DATE"] = isi_aufenthalt.apply(
#     lambda row: 0 if np.isnan(row['UPDATE_DATE']) else row['UPDATE_DATE'], axis=1)

# isi_aufenthalt["UPDATE_DATE"] = isi_aufenthalt["UPDATE_DATE"].astype(int)

In [16]:
##############################################################################
#Neue Daten speichern
##############################################################################

In [17]:
isi_stamm.to_csv("../Cleaned_Data/isi_stammdaten.csv")
isi_aufenthalt.to_csv("../Cleaned_Data/isi_aufenthalt.csv")
isi_auftragsdetails.to_csv("../Cleaned_Data/isi_auftragsdetails.csv")
isi_auftraege.to_csv("../Cleaned_Data/isi_auftraege.csv")